In [ ]:
#the usual suspects
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('ggplot')

In [ ]:
X = pd.read_csv('../input/KaggleV2-May-2016.csv')
X.isnull().values.any()


I created a label vector with the no-show outcome. Afterwards just some data wrangling.

In [ ]:
y = X['No-show']
y = y.map({'No': 0, 'Yes': 1})

X = X.drop(labels = ['No-show', 'PatientId', 'AppointmentID'], axis = 1)
X['Neighbourhood'] = X['Neighbourhood'].astype('category').cat.codes
X['Gender'] = X['Gender'].map({'M': 0, 'F': 1})

Some more data wrangling. I opted to see how many hours passed between the schelued day and the appointment. 
Data is not that good in this regard as only the day and not the time is reported in some cases.

In [ ]:
X['ScheduledDay'] = pd.to_datetime(X['ScheduledDay'])
X['AppointmentDay'] = pd.to_datetime(X['AppointmentDay'])

TimeDelta = X['ScheduledDay'] - X['AppointmentDay']
TimeDelta = TimeDelta / np.timedelta64(1, 'h')
TimeDelta = np.absolute(TimeDelta)

X = X.drop(labels = ['ScheduledDay', 'AppointmentDay'], axis = 1)
X = X.assign(TimeDelta = TimeDelta.values)


How does our data look now?

In [ ]:
X.head(5)

Create test and train.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 7)

I opted for a Random Forest Classifier as it seemed to me to be the most sensible choice.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 10, max_depth = 10)

forest.fit(X_train, y_train)

print (forest.score(X_test, y_test))

I wanted to check correlation between different features of the data set.

In [ ]:
X_4c = X.assign(No_show = y.values)

plt.imshow(X_4c.corr(), cmap = plt.cm.jet, interpolation = 'nearest')
plt.colorbar()
tick_marks = [i for i in range(len(X_4c.columns))]
plt.xticks(tick_marks, X_4c.columns, rotation = 'vertical')
plt.yticks(tick_marks, X_4c.columns)

plt.show()